In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [3]:
dataset = Planetoid(root='prob_data/Planetoid', name='Cora', transform=NormalizeFeatures())

Processing...
Done!


Get some information about dataset

In [4]:
print(f'Number of graphs: {len(dataset)}')

Number of graphs: 1


In [6]:
print(f'Number of features {dataset.num_features}')

Number of features 1433


In [7]:
print(f'Number of classes {dataset.num_classes}')

Number of classes 7


In [8]:
data = dataset[0]

In [9]:
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [10]:
print(f'Number of nodes: {data.num_nodes}')

Number of nodes: 2708


In [11]:
print(f'Number of edges: {data.num_edges}')

Number of nodes: 10556


In [12]:
print(f'Number of training nodes: {data.train_mask.sum()}')

Number of training nodes: 140


In [13]:
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')

Training node label rate: 0.05


In [14]:
print(f'Is undirected: {data.is_undirected()}')

Is undirected: True


In [15]:
print(data.x.shape)

torch.Size([2708, 1433])


In [16]:
data.x[0][:50]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [17]:
data.y

tensor([3, 4, 4,  ..., 3, 3, 3])

# Graph Neural Network for Node Classification

In [18]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [19]:
class GCN(torch.nn.Module):

    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # Init the layers
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.out = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        # First message passing layers (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Second message passing layers
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Out layer
        x = F.softmax(self.out(x), dim=1)
        return x





In [20]:
model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 16)
  (out): Linear(in_features=16, out_features=7, bias=True)
)


In [21]:
class RegularNN(torch.nn.Module):

    def __init__(self, hidden_channels):
        super(RegularNN, self).__init__()
        torch.manual_seed(42)

        # Init the layers
        self.fc1 = Linear(dataset.num_features, hidden_channels)
        self.fc2 = Linear(hidden_channels, hidden_channels)
        self.out = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        # First fully connected layers
        x = self.fc1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.train)

        # Second fully connected layers
        x = self.fc2(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Out layers
        x = F.softmax(self.out(x), dim=1)

        return x

In [22]:
regular_model = RegularNN(hidden_channels=16)
print(regular_model)

RegularNN(
  (fc1): Linear(in_features=1433, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=7, bias=True)
)


# Train and evaluate